<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<a href="./01_Introduction_Jupyter_widgets.ipynb"><< 01 - Introduction to Jupyter widgets </a><span style="float:right;"><a href="./03_voila_dashboards.ipynb"> 03 - Introduction to Voilà dashboards >></a></span>

# 2 - Combine Jupyter widgets with events
### Let's get interactive with Jupyter widgets 

* [Output widgets](#output)
* [Example - Interactive climate graph application](#climate_graph)

<hr>

#### Load required libraries

In [6]:
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt

<hr>

## <a id='climate_graph'></a>Example - Interactive climate graph application

Required libraries:
* [Plotly](https://plot.ly/) for interactive visualization
* [Widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)

#### Load required libraries

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import ee

import plotly.graph_objs as go


### Load ERA5 precipitation and 2m air temperature data from the Google Earth Engine data catalogue

#### Initialise EarthEngine

The following command initialises the EarthEngine Python API.

In [ ]:
ee.Initialize()

#### Load ERA5 monthly ImageCollection 

With the `ee.ImageCollection` constructor, you can construct a data object with data from the Earth Engine data catalogue. Let us load the ImageCollection of the monthly aggregated ERA5 reanalysis data.

In [ ]:
era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY')

#### Process mean precipitation for each month based on entire time series and build an `ImageCollection` of the resulting image list

From the entire data period (1979 to 2020), let us create the average for each month. For this we have to apply the `ImageCollection.reduce()` function.

If a reducer function is applied to an image collection, the output looses its projections information, as a collection can store images with different projections. For this reason, you have to set the projection again, after the `reduce` function was applied.

You can select and the projection information from the first image of the `ImageCollection` with the `select(i).projection()` function.

In [ ]:
era5_monthly_img = era5_monthly.limit(1).first()
collection_img_proj = era5_monthly_img.select(0).projection()
#era5_monthly_img.getInfo()

Now, you can loop over a list of 12 integer values (for each month one) and filter the specific months of each year. On the resulting list, a `reducer` function is applied and the outcome is appended to an empty list.

In [ ]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = era5_monthly.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
#img_list[0].getInfo()

As a final step, you can build an ImageCollection out of the image list with the function `ee.ImageCollection.fromImages()`.

In [ ]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
#meanMonths_collection.getInfo()

### Visualize an interactive climate graph with `Plotly`

#### Select the temperature and precipitation time-series based on a point feature 

With `ee.Geometry.Point()`, you can define a point feature with EarthEngine. Based on this point location, you can select the two ERA5 variables `total_precipitation_mean` and `mean_2m_air_temperature_mean`.

The result is a list of lists containing the the information `id`, `longitude`, `latitude`, `time` and `temperature`.

As an example we can specify the latitude and longitude values for Rome:

In [ ]:
longitude =12.5
latitude = 41.9

In [ ]:
point = ee.Geometry.Point(longitude,latitude)
tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()
t2m_point

#### Select the data series and convert the variable units

Let us select only the temperature and precipitation data series. During selection, you can directly convert the variable units:
- the precipitation values from `m` to `mm` by multiplying with 1000
- the 2m air temperature values from `K` to `degC` by substracting 273.15

In [ ]:
ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    
ydata_t2m = [row[4]-273.15 for row in t2m_point[1:]]

#### Define the plot and plot the climate graph 

For `precipitation`, we define a `barplot` and for `temperature` we define a `lineplot`. With the `data` and `layout` kwargs, we can bring everything together. 

The result is a climate graph for the specified location.

In [ ]:
tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

data = [tp,t2m]
layout = go.Layout(
        title='Climate graph at location '+ str(round(latitude,2)) + ' / '+ str(round(longitude,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm",
            dtick=20
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[min(ydata_t2m)-2,max(ydata_t2m)+2],
            dtick=5
        ),
        plot_bgcolor='white',
        legend=dict(
            orientation='h')
    )

fig = go.Figure(data=data, layout=layout)
fig

Let us define a function called `visualize_climate_graph` for the visualization, as we might want to re-use it later. But instead of `lat` and `lon`, we specify `lon.value` and `lat.value`. You will understand later why.

In [ ]:
def visualize_climate_graph():    
    point = ee.Geometry.Point(lon.value, lat.value)
    tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()

    ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    ydata_t2m = [row[4]-273.2 for row in t2m_point[1:]]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat.value,2)) + ' / '+ str(round(lon.value,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm",
            dtick=20
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[min(ydata_t2m)-2,max(ydata_t2m)+2],
            dtick=5
        ),
        plot_bgcolor='white',
        legend=dict(
            orientation='h')
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

### Combine the climate graph with `ipywidgets` to make it dynamic

Let us develop a small application where we have two different `FloatSliders` with the range of latitude and longitude ranges. As soon as the sliders are modified, we can click a button in order to updated the climate graph.

The first step is to define two `FloatSliders`:

In [ ]:
lat = widgets.FloatSlider(
    value=41.9,
    min=-90.0,
    max=90.0,
    step=0.25,
    description='Latitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal', #horizontal
    readout=True
)
lat

In [ ]:
lon = widgets.FloatSlider(
    value=12.5,
    min=-180.0,
    max=180.0,
    step=0.25,
    description='Longitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
lon

The `keys` command helps you to inspect the information the widget object holds. Let us inspect the `keys` from the slider `lon`. You see that it contains an entry value, which stores the current value displayed from the slider. In order to dynamically update a plot based on the slider entries, we have to use `lon.value` and `lat.value`.

In [ ]:
lon.keys

In [ ]:
lon.value

Now we define a function called `click()` that calls the visualization function `visualize_climate_graph()`, which returns the plotted figure.

In [ ]:
def click(b):
    fig = visualize_climate_graph()
    with out:
        clear_output(wait=True)
        fig.show()

Let us define a `Button` widget with the description `Plot climate graph`. You can add an event handler `on_click()`. The event handler calls the function `click()` which returns the plotted climate graph. 

In [ ]:
button=widgets.Button(description='Plot climate graph')
button.on_click(click)

The final step is to define an `Output` widget and to `display` all the widgets that shall be part of the application. Let us also make use of the `HBox` widget to arrange the two sliders horizontally.

In [ ]:
out=widgets.Output()
display(widgets.HBox([lat,lon]))
display(button)
display(out)

<br>

### Resources

- [Jupyter Output widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Output%20Widget.html#)

<br>

<a href="./01_Introduction_Jupyter_widgets.ipynb"><< 01 - Introduction to Jupyter widgets </a><span style="float:right;"><a href="./03_voila_dashboards.ipynb"> 03 - Introduction to Voilà dashboards >></a></span>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>